In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split # used for splitting training and testing data

torch.manual_seed(1)

In [2]:
plt.figure(figsize=(8,5))

# time steps per batch of data
seq_length = 100

time_steps = np.linspace(0, np.pi, seq_length + 1)

data = pd.read_csv('../Datasets/SectionData.csv')

x = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

<Figure size 576x360 with 0 Axes>

In [51]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, kernel_size, hidden_size, drop_prob):
        super(NeuralNetwork, self).__init__()
        self.model = nn.Sequential(
            nn.Conv1d(1, 1, kernel_size),
            nn.Flatten(),
            #nn.ReLU(),
            nn.Dropout(drop_prob),
            nn.Linear(input_size - kernel_size + 1, hidden_size[0]),
            nn.Linear(hidden_size[0], hidden_size[1]),
            nn.Linear(hidden_size[1], hidden_size[2]),
            nn.Softmax(dim=1),
        )
        #self.conv1 = nn.Conv1d(1, 1, kernel_size)
        #self.flat = nn.Flatten()
        #self.relu = nn.ReLU()
        #self.dropout = nn.Dropout(drop_prob)
        #self.fc1 = nn.Linear(input_size - kernel_size + 1, hidden_size[0])
        #self.fc2 = nn.Linear(hidden_size[0], hidden_size[1])
        #self.fc3 = nn.Linear(hidden_size[1], hidden_size[2])
        #self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        #print("X: " + str(x.shape))
        #out1 = self.conv1(x)
        #print("Out1: " + str(out1.size()))
        #outF = self.flat(out1)
        #print("OutF: " + str(out1.size()))
        #out2 = self.relu(outF)
        #print("Out2: " + str(out2.size()))
        #out3 = self.dropout(out2)
        #print("Out3: " + str(out3.size()))
        #out4 = self.fc1(out3)
        #print("Out4: " + str(out4.size()))
        #out5 = self.fc2(out4)
        #print("Out5: " + str(out5.size()))
        #out6 = self.fc3(out5)
        #print("Out6: " + str(out6.size()))
        #print(out6)
        #out7 = self.softmax(out6)
        #print("Out7: " + str(out7.size()))
        #print(out7)    
        
        out = self.model(x)
        
        return out

In [52]:
BEAT_TYPES_INDEX = {
    'N': 0,
    'L': 1,
    'R': 2,
    'A': 3,
    'V': 4,
    'F': 5,
}

def formatDataX(data):
    temp = torch.tensor(data).float()
    return temp.view(temp.size(0), 1, temp.size(1))

def formatDataY(data):
    return torch.tensor(data).float()

def preprocess(df):

    # drop ecgNum row
    df = df.drop(["ecgNum"], axis=1)
    # Classify the dependent and independent variables
    X = df.iloc[:, :-1].values
    Y = np.array([np.insert(np.zeros(5), BEAT_TYPES_INDEX[label], 1) for label in df.iloc[:, -1].values])
    
    # split the data into train, validate, test
    X_temp, X_test, Y_temp, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
    X_train, X_val, Y_train, Y_val = train_test_split(X_temp, Y_temp, test_size=0.2, random_state=0)
    
    return formatDataX(X_train), formatDataX(X_val), formatDataX(X_test), formatDataY(Y_train), formatDataY(Y_val), formatDataY(Y_test)


In [53]:
#X_train, X_val, X_test, Y_train, Y_val, Y_test = preprocess(data)
#print(X_train.size())
#print(X_val.size())
#print(X_test.size())
#print(Y_train.size())
#print(Y_val.size())
#print(Y_test.size())

In [54]:
# size of the input at each time step
# TODO: handle input sizes
input_size = 100
# kernel size
kernel_size = 5
# size of the hidden state and cell state at each time step
hidden_size = [64, 32, 6]
# dropout probability
drop_prob = 0.5

# instantiate the NN
neuralNet = NeuralNetwork(input_size, kernel_size, hidden_size, drop_prob)
print(neuralNet)

NeuralNetwork(
  (model): Sequential(
    (0): Conv1d(1, 1, kernel_size=(5,), stride=(1,))
    (1): Flatten()
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=96, out_features=64, bias=True)
    (4): Linear(in_features=64, out_features=32, bias=True)
    (5): Linear(in_features=32, out_features=6, bias=True)
    (6): Softmax(dim=1)
  )
)


In [55]:
# Mean Squared Error and Adam Optimizer with a learning rate of 0.01
# TODO: play with LR
#criterion = nn.MSELoss()
#optimizer = optim.SGD(net.parameters(), lr=0.01)
#optimizer = torch.optim.Adam(neuralNet.parameters(), lr=0.01)

In [56]:
# train the NN
def train(net, X, Y, n_steps, print_every):  
    criterion = nn.MSELoss()
    #optimizer = optim.SGD(net.parameters(), lr=0.01)
    optimizer = torch.optim.Adam(net.parameters(), lr=0.01)
    
    for batch_i, step in enumerate(range(n_steps)):
        optimizer.zero_grad()
        out = net(X)
        loss = criterion(out, Y)
        loss.backward()
        optimizer.step()
        print('Loss ', loss.item())

    return net

In [57]:
n_steps = 100
print_every = 45

X_train, X_val, X_test, Y_train, Y_val, Y_test = preprocess(data)

In [58]:
trained_NN = train(neuralNet, X_train, Y_train, n_steps, print_every)

Loss  0.13735166192054749
Loss  0.12685279548168182
Loss  0.11254105716943741
Loss  0.09052569419145584
Loss  0.07302019745111465
Loss  0.0780811682343483
Loss  0.08266469836235046
Loss  0.0842873752117157
Loss  0.08489696681499481
Loss  0.08511433005332947
Loss  0.08521134406328201
Loss  0.08513137698173523
Loss  0.08497758209705353
Loss  0.08468446880578995
Loss  0.08421865850687027
Loss  0.08333616703748703
Loss  0.08191660791635513
Loss  0.0796315148472786
Loss  0.07622276246547699
Loss  0.07294794917106628
Loss  0.07400626689195633
Loss  0.07666225731372833
Loss  0.07438042014837265
Loss  0.0716535672545433
Loss  0.07097186893224716
Loss  0.07110040634870529
Loss  0.07122339308261871
Loss  0.07064484059810638
Loss  0.06974456459283829
Loss  0.06860290467739105
Loss  0.0678168386220932
Loss  0.06798000633716583
Loss  0.06818224489688873
Loss  0.06729642301797867
Loss  0.066430002450943
Loss  0.06558962911367416
Loss  0.06522255390882492
Loss  0.06493443250656128
Loss  0.06431486457

In [59]:
def test(net, X, Y):
    results = net(X)
    resValues, resIndices = torch.max(results, 1)
    testValues, testIndices = torch.max(Y, 1)
    print(resIndices.shape)
    print(testIndices.shape)
    #for i in range(len(resIndices)):
    print(results[100])
    print(Y[100])
    print(resIndices[100])
    print(testIndices[100])
        #break
        #pass

In [60]:
test(trained_NN, X_test, Y_test)

torch.Size([20101])
torch.Size([20101])
tensor([8.9295e-01, 1.0557e-01, 7.7456e-04, 6.5290e-05, 4.7506e-06, 6.3297e-04],
       grad_fn=<SelectBackward>)
tensor([1., 0., 0., 0., 0., 0.])
tensor(0)
tensor(0)
